# Chat Vector for Korean LLM

- The paper on chat-vectors (https://arxiv.org/abs/2310.04799v2) only disclosed the idea and did not release any specific code.
- Currently, Junbeom(beomi/Llama-3-Open-Ko-8B-Instruct-preview) is deploying an instruction model that applies the chat-vector concept to a Korean model that has undergone Continual Pre-Training (CP) based on this idea.
- The idea itself is quite simple, but its implications are worth considering in various ways.
- However, it seems that the authors wrote this paper not so much from a robust theoretical foundation, but rather from a brilliant idea that they decided to experiment with, and upon experimenting, it actually worked.
- Implementation was based on the code from StableFluffy (https://github.com/StableFluffy/EasyLLMFeaturePorter).
- Due to limited memory conditions, there is a process of loading and then deleting the model.
- Since loading the model in single precision (FP32) would not allow two models within 64GB, it is necessary to load in mixed precision (FP16 or bfp16).

### Environment
- The computer used as a code server has limited resources.
- It uses an Intel 1260p CPU in an Intel NUC 12th generation.
- 64GB RAM, 20GB swap memory (as loading more than two models at once is not possible with 64GB, ample swap memory is essential).
- RTX 3090 24GB eGPU

# Chat Vector for Korean LLM

- chat-vector 논문(https://arxiv.org/abs/2310.04799v2)은 아이디어만 공개하고 구체적인 코드는 공개하지 않았다. 
- 현재 준범님이 아이디어를 통해 한글로 CP(Continual Pre-Training)된 모델에 chat-vector를 적용해서 instruction 모델을 배포하고 있다. 
- 아이디어 자체는 매우 간단하지만, 내포하고 있는 의미는 여러가지로 생각 해 볼 수 있을 듯 하다. 
- 다만 저자들도 매우 단단한 이론적 배경위에서 이 논문을 작성했다기 보단, 번뜩이는 아이디어를 가지고 실험해보고, 실험해보니 실제로 작동해서 이 논문을 작성한게 아닐까 한다. 
- StableFluffy님의 코드(https://github.com/StableFluffy/EasyLLMFeaturePorter)를 기준으로 구현해보았다.
- 제한적인 메모리 상황 때문에 모델을 로드했다가 삭제하는 과정이 있다. 
- 단정밀도(FP32)로 모델을 로드하면 64GB안에 두개 모델을 도르 할 수 없기 때문에, 반정밀도(FP16 or bfp16)으로 로드해야 한다. 

### Environment
- 코드서버로 사용하고 있는 컴퓨터의 환경이 제한적이다. 
- intel 1260p CPU를 사용하는 intel nuc 12th
- 64GB RAM, 20GB swap memory(64GB 많으로 두대 이상의 모델을 한번에 로드 해 연산을 할 수 없기 때문에, 넉넉한 swap 메모리는 필수이다.)
- rtx 3090 24GB eGPU


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#Define function

def calculate_model_diffs(model_a, model_b):
    model_a_dict = model_a.state_dict()
    model_b_dict = model_b.state_dict()
    print("model_a: ", len(model_a_dict))
    print("model_a: ", len(model_b_dict))
    model_diffs = {}
    for key in model_a_dict.keys():
        if key in model_b_dict:
            model_diffs[key] = model_a_dict[key] - model_b_dict[key]
            print(f"Diff calculated for {key}")
    print(len(model_diffs))
    return model_diffs


def apply_model_diffs(target_model, model_diffs):
    target_state_dict = target_model.state_dict()
    for key in model_diffs.keys():
        print(key)
        print(model_diffs[key])
        target_state_dict[key] += model_diffs[key]
        print(f"Diff applied for {key}")
    target_model.load_state_dict(target_state_dict)

In [3]:
# Load Models
chat_model_name = "meta-llama/Meta-Llama-3-8B-Instruct" 
base_model_name = "meta-llama/Meta-Llama-3-8B"
target_model_name = "beomi/Llama-3-Open-Ko-8B"  

In [4]:
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.bfloat16,  cache_dir = cache_model_dir)
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, torch_dtype=torch.bfloat16,  cache_dir = cache_model_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
print("Calculating model diffs...")
model_diffs = calculate_model_diffs(chat_model, base_model)
print("Model diffs calculated.")

Calculating model diffs...
model_a:  291
model_a:  291
Diff calculated for model.embed_tokens.weight
Diff calculated for model.layers.0.self_attn.q_proj.weight
Diff calculated for model.layers.0.self_attn.k_proj.weight
Diff calculated for model.layers.0.self_attn.v_proj.weight
Diff calculated for model.layers.0.self_attn.o_proj.weight
Diff calculated for model.layers.0.mlp.gate_proj.weight
Diff calculated for model.layers.0.mlp.up_proj.weight
Diff calculated for model.layers.0.mlp.down_proj.weight
Diff calculated for model.layers.0.input_layernorm.weight
Diff calculated for model.layers.0.post_attention_layernorm.weight
Diff calculated for model.layers.1.self_attn.q_proj.weight
Diff calculated for model.layers.1.self_attn.k_proj.weight
Diff calculated for model.layers.1.self_attn.v_proj.weight
Diff calculated for model.layers.1.self_attn.o_proj.weight
Diff calculated for model.layers.1.mlp.gate_proj.weight
Diff calculated for model.layers.1.mlp.up_proj.weight
Diff calculated for model.

In [7]:
del chat_model, base_model

In [8]:
target_model = AutoModelForCausalLM.from_pretrained(target_model_name, torch_dtype=torch.bfloat16,  cache_dir = cache_model_dir)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
print("Applying model diffs...")
apply_model_diffs(target_model, model_diffs)
print("Model diffs applied.")

Applying model diffs...
model.embed_tokens.weight
tensor([[-7.6294e-05,  2.7466e-04,  8.2397e-04,  ..., -2.1362e-04,
          4.7302e-04, -8.5831e-06],
        [ 5.1880e-04, -3.8910e-04, -7.6294e-06,  ...,  5.1880e-04,
          3.8147e-05,  4.9591e-04],
        [ 2.1362e-03, -4.8828e-04, -6.9046e-04,  ...,  1.8311e-04,
          3.9673e-04, -5.9509e-04],
        ...,
        [-1.0340e-25, -1.2925e-26, -1.0340e-25,  ...,  0.0000e+00,
          1.2925e-26,  1.0340e-25],
        [-1.0340e-25,  0.0000e+00,  1.0340e-25,  ..., -1.0340e-25,
          0.0000e+00,  0.0000e+00],
        [ 4.1359e-25,  0.0000e+00, -2.5849e-26,  ..., -2.5849e-26,
          0.0000e+00,  1.2925e-26]], dtype=torch.bfloat16)
Diff applied for model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
tensor([[-9.1553e-05,  2.4414e-04, -1.5259e-05,  ...,  6.4087e-04,
          0.0000e+00,  2.4414e-04],
        [-6.1035e-05,  4.8828e-04,  4.1199e-04,  ...,  6.7139e-04,
          0.0000e+00, -2.4414e-04],
        

In [11]:
print("Saving target model...")
target_model.save_pretrained('./models/Llama-3-Open-Ko-8B-meta-intruct-vector1.0')
print("Target model saved.")

Saving target model...
Target model saved.


In [12]:
del model_diffs, target_model

## Test generative

In [17]:
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"
model_path = './models/Llama-3-Open-Ko-8B-meta-intruct-vector1.0'

In [18]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map=device_map)

tokenizer = AutoTokenizer.from_pretrained(target_model_name, cache_dir=cache_model_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
messages = [
    {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
    {"role": "user", "content": "피보나치 수열이 뭐야? 그리고 피보나치 수열에 대해 파이썬 코드를 짜줘볼래?"},
]

In [20]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [21]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
)

In [22]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Hello! 피보나치 수열은 수학에서 매우 중요한 개념입니다. 피보나치 수열은 자연수의 집합에서 시작하여 무한히 이어지는 수열을 말합니다. 각 항은 이전 항의 다음 항의 약 두 배가 되는 성질을 갖는 수열입니다. 예를 들어, 1, 2, 4, 8, 16, 32, 64, 128, 256,...와 같은 수열입니다. 이 수열은 가장 기본적인 예이지만, 이보다 더 많은 예가 있습니다. 이 수열은 수학, 컴퓨터 과학, 경제학, 물리학 등 다양한 분야에서 사용됩니다.

파이썬 코드를 짜보겠습니다. Python으로 피보나치 수열을 구현하는 예제를 보여드리겠습니다.

```
python
def fibonacci(n):
    a, b = 1, 1
for i in range(1, n+1):
    a, b = b, a + b
print(fibonacci(n))
```
위 코드는 n번째 피보나치 수열의 항을 구현합니다. 이 코드를 실행하면, 1부터 시작하여 피보나치 수열의 각 항을 출력합니다. 예를 들어, `fibonacci(10)`을 실행하면, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 985, 1597, 2584, 4181,...를 출력합니다. 이 코드는 피보나치 수열의 기본적인 예를 보여주는 예시입니다. 이 코드를 응용하여 다양한 분야에서 사용할 수 있습니다. 예를 들어, 금융 분야에서는 투자 분석에 사용할 수 있습니다. 이 수열은 성장률을 예측하는 데 사용됩니다. 물리학에서는 나비의 이동 경로를 모델링하는 데 사용할 수 있습니다. 이 수열은 매우 유용합니다.

이제까지 피보나치 수열에 대해 설명했습니다. 더 궁금한 점이 있으신가요?


In [23]:
print(len(outputs[0]))

555


In [24]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
)

In [25]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

피보나치 수열(피보나치 수열, Fibonacci sequence)은 수학에서 가장 유명한 수열 중 하나입니다. 피보나치 수열은 0과 1로 시작하여 다음 항은 이전 항의 합으로 구성되는 수열입니다. 예를 들어, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,...으로 계속됩니다. 피보나치 수열은 수학, 컴퓨터 과학, 경제학, 생물학 등 다양한 분야에서 응용됩니다.

피보나치 수열을 파이썬 코드로 구현하는 방법은 다음과 같습니다.
```python
a = 0
b = 1
fib_sequence = [a, b]
while True:
    a, b = b, a + b
    print(fib_sequence)
```
위 코드는 피보나치 수열을 생성하는 파이썬 코드입니다. 이 코드는 0과 1로 시작하여 다음 항은 이전 항의 합으로 구성되는 피보나치 수열을 생성합니다. 이 코드를 실행하면 피보나치 수열이 출력됩니다. 이 코드는 수학자 피보나치의 이름에서 따온 이름입니다. 피보나치는 이 수열을 발견한 이탈리아 수학자입니다.

이 코드를 실행하면 피보나치 수열이 출력됩니다. 이 코드는 수학자 피보나치의 이름에서 따온 이름입니다. 피보나치는 이 수열을 발견한 이탈리아 수학자입니다. 이 수열은 수학, 컴퓨터 과학, 경제학, 생물학 등 다양한 분야에서 응용됩니다. 이 수열은 수학자 피보나치의 이름에서 따온 이름입니다. 피보나치는 이 수열을 발견한 이탈리아 수학자입니다. 이 수열은 수학, 컴퓨터 과학, 경제학, 생물학 등 다양한 분야에서 응용됩니다. 이 수열은 수학자 피보나치의 이름에서 따온 이름입니다. 이 수열은 수학, 컴퓨터 과학, 경제학, 생물학 등 다양한 분야에서 응용됩니다. 이 수열은 수학자 피보나치의 이름에서 따온 이름입니다. 이 수열은 수


In [26]:
print(len(outputs[0]))

598


In [27]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=0.95,
)

In [28]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

의아한 친구네! 피보나치 수열(Feigenbaum sequence)은 19세기 독일 수학자 레온하르트 오일러의 연구 결과물로, 1863년에 발표된 수열이야. 피보나치 수열은 0과 1의 재귀적 수열로, 각 각이 다음 수식에 의해 생성된다: a(n) = a(n-1) + a(n-2). 이 수열은 0과 1의 무한한 반복으로 구성되는데, 각 항의 값이 다음 항의 값에 따라 결정되어. 다음은 예를 들어보자. 0, 1, 0, 1, 0, 1, 0, 1,....

이 수열을 파이썬 코드로 작성하려면 다음과 같이 짤 수 있어요!
```
a = 0
for i in range(10):
  print(a(i % 2) if i % 2 == 0 then
  print(1)
  else
  print(0)
```
이 코드는 수열의 각 항을 출력할 때마다 0 또는 1을 출력하는 거야. 이 코드를 실행하면, 0과 1의 피보나치 수열이 출력돼!
이 수열은 특정 패턴이 없는 것 같지만, 실제로는 패턴이 숨어있어. 그걸 찾는 것이 중요한 거야.
원래 피보나치 수열은 암호 통신에 사용돼. 예를 들어 코드를 인코딩할 때, 이 수열을 사용하면 코드의 길이를 줄이거나 복잡도를 낮추는 데 사용하잖아!.
이해했나? 다른 질문은 더 있어?


In [29]:
print(len(outputs[0]))

464
